# Constraint Manifold Sampling

In order to provide sufficient sampled data for allowing a surrogate model to learn the boundaries that define the satisfaction of constraints in an N-dimensional decision space, an iterative method is proposed. This method is only applicable to continuous non-linear equations, from which the constraint satisfaction is determined.  
<br>

In [ ]:
%matplotlib inline
%cd "~/lab/NB10422645"

import matplotlib.pyplot as plt
from src.notebooks.constraint_manifold_sampling import *
import sympy as sp
import numpy as np

## Demonstration in $\mathbb{R}^2$

Assume the following arbitrary manifold describing the region within which an arbitrary constraint is no longer satsified according to an equation in $\mathbb{R}^2$.

![Fig 1: Manifold depiction in $\mathbb{R}^2$, where white regions depict constraint satisfaction.](graphics/constraint_1.png)

### Definition of Constraint equation

According to the previous $\mathbb{R}^2$ example; $\overline{X}=[{x_1,x_2}]$. The Inequality constrain satisfaction is defined as $\mathcal{C}(\overline{X})\leq{0}$. The manifold at which we would like to arbitrary points to converge towards is defined by $\mathcal{C}(\overline{X})={0}$. Figure 1 is described by the following domain:

\begin{equation}
    \begin{aligned}
    x_1 &\in [0,1] \\
    x_2 &\in[0,1] \\
    \end{aligned}
    \label{eq:domain}\tag{1}
\end{equation}

Let the regions shown in Figure 1 be described as points that must be outside of two independent circular regions. The constraint equation would then be defined as:

\begin{equation}
    \mathcal{C}_{circ}(\overline{X}) = R - \sqrt{(x_1-x_{1_0})^2 + (x_2 - x_{2_0})^2}
    \label{eq:c_circ}\tag{2}
\end{equation}

where R and $[x_{1_0}, x_{2_0}]$ and are the radius and location of the center of the circle respectively.

### Linearisation of Constraint equation

The constraint equation is linearised about $\overline{X}_0$ through the use of Taylor expansion, described in matrix notation as follows:

\begin{equation}
    \overline{y}=A(\overline{X}) = A(\overline{X}_0) + A'(\overline{X}_0)(\overline{X}-\overline{X}_0) +\;...\;= A(\overline{X}_0) + A'(\overline{X}_0)\Delta{\overline{X}} +\;...\;
    \label{eq:taylor}\tag{3}
\end{equation}

Equation \ref{eq:taylor} can be rewritten in the first order accurate form:

\begin{equation}
    \overline{y}^* - \overline{y}_0 = \Delta{\overline{y}} = A(\overline{X}) - A(\overline{X}_0) = A'(\overline{X}_0)\Delta{\overline{X}}
    \label{eq:taylor2}\tag{4}
\end{equation}

Equation \ref{eq:taylor2} describes a first-order accurate, iterative equation which can be used to provide update steps ($\Delta{\overline{X}}$) to $\overline{X}_0$ in order for $\overline{X}_0$ to converge towards a reference value of $\overline{y}^*$. Comparing Equation 2 to the prescribed linearisation, we set the following: 

\begin{equation}
    \begin{aligned}
    \\
    \overline{y}_0     & = \mathcal{C}_{obs}(\overline{X}_0) \\
    \\
    A'(\overline{X}_0) & = \frac{\partial{\mathcal{C}_{circ}}}{\partial{\overline{X}}}\bigg|_{\overline{X}_0} = \mathbf{J}_{\overline{X}}|_{\overline{X}_0}
    \end{aligned}
\end{equation}

### Algorithm definition

```
DEFINITIONS
---------------------------------------
y* : reference value for convergence
y_obs : observed value for current X
Jx : jacobian matrix w.r.t. X
X : decision matrix
X_sampled : sampled values for init
tol : tolerance setting for convergence
maxiter : maximum number of iterations
---------------------------------------

delta_y = y* - y_obs
X = X_sampled 
count = 0

while (|delta_y| > tol) and (count <= maxiter):
    count = count + 1
    
    # Moor penrose inverse used here.
    delta_X = (J.T * J)^(-1) * J.T * delta_y
    
    # Update X
    X = X + delta_X
    
    # Recalculate current observed y for new X
    y_obs = f(X)
```

### Example use cases

#### Convergence to Manifold ($\overline{y}^* = 0$)

Setting the $y^*$ value results in the convergence to the surface of the constraint manifold, as seen for the two dimensional case seen below:

In [ ]:
fig = plt.figure(figsize=(3,3), dpi=200)
ax = fig.add_subplot(111)
simple_2d_case_study(ax, maxiter=1, k=1, manifold=True, y_ref=0)

#### Convergence to layers Manifold offsets ($\overline{y}^* = \{-0.10, -0.05, 0.05, 0.10\}$)

Samples can be obtained from offset layers of the manifold, in order to provide more interior and exterior information with respect to the constraint manifold, as seen in the example below: 

In [ ]:
fig = plt.figure(figsize=(3,3), dpi=200)
ax = fig.add_subplot(111)
simple_2d_case_study(ax, maxiter=1, k=1, manifold=False, y_ref=[-0.10, -0.05, 0.05, 0.10])

### Convexity & concavity of the cost function

Introducing a new exponent variable to the original constraint equation for $\mathcal{C}_{circ}(\overline{X})$, allows us to modify the concavity and convexity of the constraint equation which is in $\mathbb{R}^3$.

\begin{equation}
    \mathcal{C}_{circ}(\overline{X}) = R^{k} - [(x_1-x_{1_0})^2 + (x_2 - x_{2_0})^2]^{k/2}
    \label{eq:c_circ}\tag{2}
\end{equation}

It should be noted that the previous examples were such that $k=1$, which allowed for a single iteration for the previous results obtained. This suggests that the equation with $k=1$ is first-order dominant. The following example shows the case with $k=2$, which increases the concavity of the equation. The left and right images are the first and second iterations towards the manifold, respectively.

In [ ]:
fig = plt.figure(figsize=(6,3), dpi=200)
ax = fig.add_subplot(121)
simple_2d_case_study(ax, maxiter=1, k=2, manifold=True, y_ref=0)
ax = fig.add_subplot(122)
simple_2d_case_study(ax, maxiter=2, k=2, manifold=True, y_ref=0)

In the case of $k=0.5$, with increased convexity, ceteris paribus:

In [ ]:
fig = plt.figure(figsize=(6,3), dpi=200)
ax = fig.add_subplot(121)
simple_2d_case_study(ax, maxiter=1, k=0.5, manifold=True, y_ref=0)
ax = fig.add_subplot(122)
simple_2d_case_study(ax, maxiter=2, k=0.5, manifold=True, y_ref=0)

## 2D Attitutude problem: $\mathcal{C}_{\Omega_{max}}$

The objective is to sample closely to the manifold in order to provide sample data that clearly distinguishes the manifold at which the constraint ($\mathcal{C}$) is no longer satisfied. The following equation describes a manifold in the $\mathbb{R}^{2N-1}$ formulation of the 2D attitude problem.  
<br>

\begin{equation}
\bigg|\frac{4\pi{n}-2\mathbb{S}_{\Omega}-\Omega_{0}t_{w,0}-\Omega_{N}t_{w,N}}{t_{w,0}+t_{w,N}}\bigg|\leq{\Omega_{max}}
\label{eq:1}
\end{equation}  
<br>

where $\mathbb{S}_{\Omega}$ is the trapezoidal sum of the interior values of the sequence of $\Omega$ values. The equation describes the condition for the periodic domain, as $\Omega_{1,...,N}$ are sampled within the constraint boundary of $\mathcal{C}_{\Omega_{max}}$, $\Omega_0$ is the only angular rate value that can cause $\mathcal{C}_{\Omega_{max}}$ to be not satisfied.


Taking the case where there exists two interior $\Omega$ values, the input space is $\mathbb{R}^{5}$. This constraint can can be expressed in the non-linear form $\overline{y}=A(\overline{X})$, where:

\begin{equation}
    \overline{X} = \begin{matrix}[\;\Omega_1 & \Omega_2 & t_{w,0} & t_{w,1} & t_{w,2}\end{matrix}]^T
\end{equation}

\begin{equation}
    A(\overline{X}) = \Omega_0(\overline{X}) = 
    \begin{matrix}\bigg[
    \frac{4\pi{n} - (\Omega_1 + \Omega_2)\cdot{}t_{w,1} + \Omega_0{t_{w,0}} + \Omega_2{t_{w,2}}}{t_{w,0} + t_{w,N}}
    \end{matrix}\bigg]
\end{equation}

This can be reformulated into a taylor expansion about $\overline{X}_0$:

\begin{equation}
    y=A(\overline{X}) = A(\overline{X}_0) + A'(\overline{X}_0)(\overline{X}-\overline{X}_0) +\;...\;= A(\overline{X}_0) + A'(\overline{X}_0)\Delta{\overline{X}} +\;...\;
\end{equation}

This can be reformulated to the first order accurate equation:

\begin{equation}
    \Delta{\Omega_0} = \frac{\partial{\Omega_0}}{\partial{\overline{X}}}\bigg|_{\overline{X}_0} \Delta{\overline{X}}
\end{equation}

Taking the partial differential of the equation provides the following:

\begin{equation}
    \frac{\partial{\Omega_0}}{\partial{\overline{X}}} = \mathbf{J}_{\Omega_{0}}
\end{equation}

In [ ]:
# Define symbols.
Omega0  = sp.Symbol("Omega_0")
Omega1  = sp.Symbol("Omega_1")
Omega2  = sp.Symbol("Omega_2")
tw0     = sp.Symbol("t_w_0")
tw1     = sp.Symbol("t_w_1")
tw2     = sp.Symbol("t_w_2")
n       = sp.symbols("n")
o_lim   = sp.symbols("\Omega_{lim}")
c_omega = sp.Symbol("\mathcal{C}_{\Omega_{max}}(\overline{X})")
x_vec   = sp.Matrix([Omega1, Omega2, tw0, tw1, tw2])

# Abs omega max manifold for Omega0.
abs_omega_max = 0.002

# r.h.s. of constraint equation.
c_omega_rhs = o_lim - (4 * sp.pi * n - 2 * ((Omega1 + Omega2)/2 * tw1) - Omega1 * tw0 - Omega2 * tw2) / (tw0 + tw2)

# Define equation.
c_omega_eq = sp.Eq(c_omega, c_omega_rhs)

# Jacobian of constraint observation.
Jx = sp.Matrix([c_omega_eq.rhs]).jacobian(x_vec)

# Define auxilliary function form of Jacobian.
Jx_func = sp.lambdify([Omega1, Omega2, tw0, tw1, tw2, n, o_lim], Jx, "numpy")

# Define auxilliary function form of y_observation.
c_omega_func = sp.lambdify([Omega1, Omega2, tw0, tw1, tw2, n, o_lim], c_omega_rhs, "numpy")

# Define bounds.
bounds = [[-1 * abs_omega_max] * 2 + [0.001] * 3,
          [+1 * abs_omega_max] * 2 + [1.000] * 3]


c1_o = System(c_omega_func, Jx_func, [1, -abs_omega_max])

c2_o = System(c_omega_func, Jx_func, [1, +abs_omega_max])

sample_x_grid_2 = sampled_grid(bounds, 15, flatten=True)

c1_result = iterate2manifold(bounds, sample_x_grid_2, c1_o.y, c1_o.Jx, 
                            maxiter=5, force_bounds=False, tol=1e-6, verbose=True)

c2_result = iterate2manifold(bounds, sample_x_grid_2, c2_o.y, c2_o.Jx, 
                            maxiter=5, force_bounds=False, tol=1e-6, verbose=True)